# Project 2 — IT Ticket Triage Bot
**Track:** AI Automation Parallel Track  
**Date Started:** 2026-02-25  
**Status:** 🔵 In Progress

---

## Goal
Build a tool that reads an IT support ticket and uses Claude to return a structured triage assessment:
severity level, category, impact, first response, and next step.

## What I expect to learn
- How to write a **system prompt** that controls Claude's behavior
- How to ask Claude for **structured output** in a specific format
- How to **parse** Claude's text response into a Python dictionary
- How to wrap logic into a **function** (`def triage_ticket(...)`)
- How to read input from a **.txt file** (file I/O)
- How to use **f-strings** to build dynamic prompts
- How to loop over files in a folder (**batch processing**)

---
## Step 1 — Install Dependencies
Same packages as Project 1. Skip if already installed.

In [ ]:
%pip install anthropic python-dotenv

---
## Step 2 — Load API Key

Your `.env` file is now at the **repo root** (one level above the `projects/` folder):
```
ai-automation-track/
├── .env                  ← shared API key lives here
├── projects/
│   ├── p1-hello-claude/
│   └── p2-ticket-triage/  ← you are here
```

We need to walk **up** the directory tree to find it.

> **Why shared?** Every project needs the same key. One `.env` at the root means you never duplicate it — and never accidentally commit it.

In [ ]:
import os
import anthropic
from pathlib import Path
from dotenv import load_dotenv

# Walk up from this notebook's location to find the .env file
notebook_dir = Path().resolve()
for directory in [notebook_dir] + list(notebook_dir.parents):
    env_file = directory / ".env"
    if env_file.exists():
        load_dotenv(env_file)
        print(f"✅ Found .env at: {env_file}")
        break

# Verify key loaded
api_key = os.getenv("ANTHROPIC_API_KEY")
if api_key:
    print(f"✅ API key loaded — starts with: {api_key[:10]}...")
else:
    print("❌ API key not found — check your .env file")

client = anthropic.Anthropic(api_key=api_key)

**What I observed:**  
*(Did it find the .env? Which directory did it find it in?)*

- 
- 

---
## Step 3 — Write the System Prompt

In Project 1 you learned that a system prompt gives Claude a **role**.

In this project the system prompt does two things:
1. Gives Claude the role of an IT triage specialist
2. **Forces a specific output format** — so you can parse it reliably in Python

This is the most important skill in this project. The format you define in the prompt is exactly what you'll parse in Step 5.

In [ ]:
SYSTEM_PROMPT = """You are an expert IT support triage specialist with 15 years of experience
in enterprise environments including financial services, hedge funds, and fintech companies.

Your job is to analyze incoming IT support tickets and return a structured triage assessment.

Always respond in this EXACT format — nothing before or after, no extra text:

SEVERITY: [LOW / MEDIUM / HIGH / CRITICAL]
CATEGORY: [Identity & Access / AV & Conference / Network & VPN / Hardware / Software / Endpoint / Printing / Other]
IMPACT: [brief description of who is affected and business impact]
FIRST RESPONSE: [what to tell the user right now — be specific and actionable]
NEXT STEP: [what IT should do immediately after sending the first response]

Severity guide:
- CRITICAL: Active business disruption, executive impact, trading desk, or time-sensitive meeting in progress
- HIGH: Single user fully blocked from doing their job, or multiple users moderately affected
- MEDIUM: User partially blocked, workaround exists, or issue is not time-sensitive
- LOW: Minor inconvenience, question, or request that can be scheduled

Be concise. Each field should be 1-2 sentences max."""

print("System prompt ready.")
print(f"Length: {len(SYSTEM_PROMPT)} characters")

**Think about this:**  
*(Why do we need to specify the exact format in the prompt? What would happen if we didn't?)*

- 
- 

---
## Step 4 — Make the First Triage Call

Let's send a ticket and look at what Claude returns — **raw, before any parsing**.

Notice the f-string in the user message: `f"Triage this IT support ticket:\n\n{ticket_text}"`  
This is how you inject dynamic content into a prompt.

In [ ]:
# A realistic ticket — the kind you'd see at a hedge fund
ticket_text = "User can't log into Okta. Getting stuck in an MFA loop — it sends the push notification, she approves it, then it immediately asks for MFA again. Affects the trading desk. She has a market open in 10 minutes."

# Make the API call
response = client.messages.create(
    model="claude-haiku-4-5-20251001",
    max_tokens=512,
    system=SYSTEM_PROMPT,
    messages=[
        {"role": "user", "content": f"Triage this IT support ticket:\n\n{ticket_text}"}
    ]
)

# Print the raw response text
raw_output = response.content[0].text
print("=== RAW RESPONSE FROM CLAUDE ===")
print(raw_output)

**What I observed:**  
*(Did Claude follow the format exactly? What severity did it assign? Does the response make sense given your IT background?)*

- 
- 

---
## Step 5 — Parse the Response into a Dictionary

Claude returned plain text. To use it in code you need to convert it into a Python **dict** — a data structure with named keys.

The strategy: loop over each line, check if it starts with a known prefix (`SEVERITY:`, `CATEGORY:`, etc.), and extract the value after the colon.

**New Python concept: dictionaries**
```python
result = {"severity": "CRITICAL", "category": "Identity & Access"}
print(result["severity"])  # → CRITICAL
```

In [ ]:
# Map each line prefix to a dictionary key
field_map = {
    "SEVERITY:": "severity",
    "CATEGORY:": "category",
    "IMPACT:": "impact",
    "FIRST RESPONSE:": "first_response",
    "NEXT STEP:": "next_step"
}

# Start with an empty result dict
result = {"severity": "", "category": "", "impact": "", "first_response": "", "next_step": ""}

# Loop over each line in the response
for line in raw_output.strip().splitlines():
    for prefix, key in field_map.items():
        if line.startswith(prefix):
            result[key] = line[len(prefix):].strip()  # grab everything after the prefix

# Print the parsed result
print("=== PARSED RESULT ===")
for key, value in result.items():
    print(f"  {key.upper():15} : {value}")

**What I observed:**  
*(Did all 5 fields parse correctly? Can you access `result["severity"]` and get back `CRITICAL`?)*

- 
- 

In [ ]:
# Try accessing individual fields — this is the power of a dict
print("Severity:", result["severity"])
print("Category:", result["category"])
print()

# Now you can write logic based on the result
if result["severity"] == "CRITICAL":
    print("🚨 CRITICAL ticket — escalate immediately!")
elif result["severity"] == "HIGH":
    print("⚠️  HIGH severity — respond within 1 hour")
else:
    print("📋 Standard priority — add to queue")

**What I observed:**  
*(Did the if/elif logic fire correctly based on severity? This is how you'd route tickets in a real system.)*

- 
- 

---
## Step 6 — Wrap It in a Function

Right now the code runs top to bottom once. A **function** lets you reuse the same logic on any ticket.

**New Python concept: functions with type hints**
```python
def function_name(parameter: type) -> return_type:
    # code here
    return result
```

Type hints (`str`, `dict`) are optional but they document what goes in and what comes out.

In [ ]:
def parse_response(raw: str) -> dict:
    """Parses Claude's structured text response into a Python dictionary."""
    field_map = {
        "SEVERITY:": "severity",
        "CATEGORY:": "category",
        "IMPACT:": "impact",
        "FIRST RESPONSE:": "first_response",
        "NEXT STEP:": "next_step"
    }
    result = {"severity": "", "category": "", "impact": "", "first_response": "", "next_step": ""}
    for line in raw.strip().splitlines():
        for prefix, key in field_map.items():
            if line.startswith(prefix):
                result[key] = line[len(prefix):].strip()
    return result


def triage_ticket(ticket_text: str) -> dict:
    """Sends a ticket to Claude for triage. Returns a dict with severity, category, impact, etc."""
    message = client.messages.create(
        model="claude-haiku-4-5-20251001",
        max_tokens=512,
        system=SYSTEM_PROMPT,
        messages=[
            {"role": "user", "content": f"Triage this IT support ticket:\n\n{ticket_text}"}
        ]
    )
    raw = message.content[0].text
    return parse_response(raw)


print("Functions defined. Testing triage_ticket()...")

# Test it
test_result = triage_ticket("The office printer on the 12th floor is showing offline for everyone. About 15 people affected.")

print(f"Severity : {test_result['severity']}")
print(f"Category : {test_result['category']}")
print(f"Impact   : {test_result['impact']}")

**What I observed:**  
*(The printer ticket — what severity did it get? Does it feel right to you as an IT professional?)*

- 
- 

---
## Step 7 — Read a Ticket from a File

Real tickets don't come from variables — they come from files, databases, or APIs.

**New Python concept: file I/O**
```python
with open("path/to/file.txt", "r", encoding="utf-8") as f:
    content = f.read()
```
The `with` statement automatically closes the file when done — always use it.

In [ ]:
# Read ticket1.txt from the sample_tickets folder
ticket_path = "sample_tickets/ticket1.txt"

with open(ticket_path, "r", encoding="utf-8") as f:
    ticket_from_file = f.read().strip()  # .strip() removes leading/trailing whitespace

print("Ticket read from file:")
print(ticket_from_file)
print()

# Now triage it
result = triage_ticket(ticket_from_file)

print("-" * 60)
print(f"  SEVERITY   : {result['severity']}")
print(f"  CATEGORY   : {result['category']}")
print(f"  IMPACT     : {result['impact']}")
print(f"  FIRST RESP : {result['first_response']}")
print(f"  NEXT STEP  : {result['next_step']}")
print("-" * 60)

**What I observed:**  
*(Did the file read correctly? Did triage_ticket() work on file input the same as typed text?)*

- 
- 

---
## Step 8 — Test All 5 Sample Tickets

Now triage each of the 5 sample tickets one by one. Read the output and check: does the severity feel right based on your real-world experience?

In [ ]:
sample_tickets = [
    ("ticket1.txt", "Okta MFA loop — trading desk, 10 mins to market open"),
    ("ticket2.txt", "Zoom Room down — board meeting in 20 mins"),
    ("ticket3.txt", "VPN failure — new hire, working from home"),
    ("ticket4.txt", "Shared drive access denied — report due today"),
    ("ticket5.txt", "Printer offline — 15 users on 12th floor"),
]

for filename, description in sample_tickets:
    with open(f"sample_tickets/{filename}", "r", encoding="utf-8") as f:
        text = f.read().strip()
    
    result = triage_ticket(text)
    
    print(f"\n[ {filename} ] {description}")
    print(f"  Severity : {result['severity']}")
    print(f"  Category : {result['category']}")

**What I observed:**  
*(Look at the severities across all 5. Does the ranking make sense? Which one surprised you? Any you'd disagree with based on your IT experience?)*

- 
- 
- 

---
## Step 9 — Batch Mode: Loop Over a Folder

A real IT team would run this against an entire inbox of tickets — not one at a time.

**New Python concepts:**
- `os.listdir(folder)` — list all files in a folder
- `sorted()` — sort them alphabetically
- `os.path.join(folder, filename)` — safely build a file path

In [ ]:
def run_batch(folder: str):
    """Triages all .txt files in a folder and prints a summary table."""
    txt_files = sorted([f for f in os.listdir(folder) if f.endswith(".txt")])
    
    if not txt_files:
        print(f"No .txt files found in '{folder}'")
        return
    
    print(f"Triaging {len(txt_files)} tickets...\n")
    print(f"{'FILE':<15} {'SEVERITY':<12} {'CATEGORY'}")
    print("-" * 60)
    
    for filename in txt_files:
        filepath = os.path.join(folder, filename)
        with open(filepath, "r", encoding="utf-8") as f:
            ticket_text = f.read().strip()
        
        result = triage_ticket(ticket_text)
        print(f"{filename:<15} {result['severity']:<12} {result['category']}")


# Run it
run_batch("sample_tickets")

**What I observed:**  
*(This is what a real automated triage pipeline looks like. What do you think of the summary table format? How would you improve it for a real team?)*

- 
- 

---
## Step 10 — Experiment: Change the System Prompt

The system prompt controls everything. Let's test what happens when you change the severity definitions.

Try making the bot **more aggressive** about severity — so more tickets come back CRITICAL.

In [ ]:
# Experiment: stricter severity definitions
STRICT_PROMPT = """You are an IT triage bot at a financial services firm where downtime = lost money.
Err on the side of escalation. When in doubt, go higher on severity.

Always respond in this EXACT format:

SEVERITY: [LOW / MEDIUM / HIGH / CRITICAL]
CATEGORY: [Identity & Access / AV & Conference / Network & VPN / Hardware / Software / Endpoint / Printing / Other]
IMPACT: [who is affected and business risk]
FIRST RESPONSE: [what to tell the user]
NEXT STEP: [what IT does next]"""

# Test the same printer ticket with both prompts
printer_ticket = "The office printer on the 12th floor is showing offline for everyone. About 15 people affected."

# Original prompt
r1 = client.messages.create(
    model="claude-haiku-4-5-20251001", max_tokens=256, system=SYSTEM_PROMPT,
    messages=[{"role": "user", "content": f"Triage this: {printer_ticket}"}]
)
parsed1 = parse_response(r1.content[0].text)

# Strict prompt
r2 = client.messages.create(
    model="claude-haiku-4-5-20251001", max_tokens=256, system=STRICT_PROMPT,
    messages=[{"role": "user", "content": f"Triage this: {printer_ticket}"}]
)
parsed2 = parse_response(r2.content[0].text)

print("=== ORIGINAL PROMPT ===")
print(f"Severity: {parsed1['severity']}  |  Category: {parsed1['category']}")

print("\n=== STRICT PROMPT ===")
print(f"Severity: {parsed2['severity']}  |  Category: {parsed2['category']}")

**What I observed:**  
*(Did the severity change between the two prompts? What does this tell you about how much control you have over AI behavior through prompting?)*

- 
- 

---
## Step 11 — Connect Back to ticket_triage.py

Everything you built in this notebook lives in `ticket_triage.py` — the production-ready script.

Run it here directly from the notebook to see the same output you'd get from the command line.

In [ ]:
# Run the final script on a single ticket
!python ticket_triage.py sample_tickets/ticket2.txt

In [ ]:
# Run batch mode on all sample tickets
!python ticket_triage.py --batch sample_tickets/

---
## Results Summary

*(Fill this in when you finish)*

**What I built:**  
An IT ticket triage bot that reads a support ticket (text or .txt file), sends it to Claude with a structured system prompt, and returns a parsed triage assessment as a Python dictionary.

**What worked well:**  
- 
- 

**What confused me / questions I still have:**  
- 
- 

**Key insight about system prompts (in my own words):**  
*(What did you learn about how system prompts control AI output?)*



**Key insight about structured output (in my own words):**  
*(Why does it matter that you define the format in the prompt?)*



**If I were explaining this to a coworker, I'd say:**  
*(Write 2-3 sentences like you're explaining it at a team meeting)*



**Time spent:** ___ hours

**GitHub:** Pushed ✅ / Not yet ⬜

---
**Next project:** P3 — AI Document Summarizer